In [ ]:
%matplotlib qt
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import numpy as np
import random
import copy
import math

fig, ax = plt.subplots()
xdata, ydata = [], []
ln, = plt.plot([], [], 'r-', animated=True)

def init():
    ax.set_xlim(0, 1000)
    ax.set_ylim(0, get_score(a) * 2)
    return ln,

def update(frame):
    xdata.append(frame)
    ydata.append(best_scores[frame])
    ln.set_data(xdata, ydata)
    return ln,




def is_valid_individual(individual):
    """
    判断个体是否满足约束条件
    """
    enter_time = calculate_enter_time_individual(individual)
    if any(et[-1] > 300 for et in enter_time):
        return False
    if any(et[find_target(ind, 1)[0]] > 240 for ind, et in zip(individual, enter_time)):
        return False
    total_third_element = 0
    count = 0
    for sublist in individual:
        for item in sublist:
            val = item[2]
            if val < 1000/60 or val > 3000/60:
                return False
            total_third_element += val
            count += 1
    avg_third_element = total_third_element / count
    if avg_third_element > 2000/60:
        return False
    return True

def calculate_enter_time_individual(path):
    individual = path.copy()
    individual[1] = [[0, 0, 0]] + individual[1]
    individual[0] = [[0, 10, 0]] + individual[0]
    individual[2] = [[0, 20, 0]] + individual[2]
    timetable = {i: 0 for i in range(1, 8)}
    timetable[7] = float('inf')

    n = len(individual)
    result = [[0] for _ in range(n)]
    variables = {f"tourist_{i + 1}": 1 for i in range(n)}
    while(variables['tourist_1'] <=7 or variables['tourist_2'] <=7 or variables['tourist_3'] <=7):
        time = float("inf")
        ind = 0
        for i in range(len(result)):
            site = variables[f"tourist_{i+1}"]
            if site ==8:
                continue
            current_node = individual[i][site][0]
            previous_node = individual[i][site - 1][0]
            arrive_time = result[i][-1] + adj_matrix[previous_node][current_node]/individual[i][site][2] + individual[i][site-1][1]
            if arrive_time < time:
                ind = i
                time = arrive_time
        site = variables[f"tourist_{ind+1}"]
        arrive_time = time
        index = individual[ind][site][0]
        if index == 3:
            if arrive_time < timetable[index] - 1e-9:
                arrive_time = timetable[index]
                timetable[index] = arrive_time + individual[ind][site][1]
            elif abs(round(arrive_time / 30) * 30 - arrive_time)> 1e-9 and arrive_time > timetable[index] + 1e-9:
                    num = copy.deepcopy(arrive_time)
                    integer_part = int(num)
                    decimal_part = num - integer_part
                    remainder = integer_part % 30
                    difference = 30 - remainder - decimal_part

                    timetable[index] = individual[ind][site][1] + difference + arrive_time
                    arrive_time = difference + arrive_time                                    
            else:
                timetable[index] = timetable[index] +30
        elif index == 7:
            arrive_time = arrive_time
        else:
            if arrive_time > timetable[index]:
                 timetable[index] = arrive_time + individual[ind][site][1]
            else:
                 arrive_time = timetable[index]
                 timetable[index] = timetable[index] + individual[ind][site][1]               
        result[ind].append(arrive_time)
        variables[f"tourist_{ind+1}"] = variables[f"tourist_{ind+1}"] +1       
    return result

def find_target(nested_list, target):
    # 将二级列表转换为NumPy数组
    nested_array = np.array(nested_list)
    
    # 获取所有子列表的第一个元素
    first_elements = nested_array[:, 0]
    
    # 使用NumPy的索引功能来查找目标数字
    indices = np.where(first_elements == target)[0]
    
    if len(indices) == 0:
        print("Target not found! Nested List:", nested_list, "Target:", target)
    
    return indices

def get_score(individual):
    """
    评价一个个体
    """
    sightsee = 0
    entertime = calculate_enter_time_individual(individual)
    for i in range(len(entertime)):
        sightsee = sightsee + (330 - entertime[i][-1])
    for ind in range(len(individual)):
        for i in range(len(individual[ind])-1):
            sightsee = sightsee + individual[ind][i][1]
    return sightsee

def modify_element(element, range_dict):
    # Modify the second value
    idx_val = element[0]
    range_val = range_dict[idx_val]
    if isinstance(range_val, tuple):
        element[1] = np.random.randint(range_val[0], range_val[1]+1)
    else:
        element[1] = range_val
    
    # Modify the third value
    element[2] = np.random.uniform(1000/60, 3000/60)
    
    return element

def get_valid_neighbor(a):
    while True:
        # Create a shallow copy since we'll be modifying the original structure
        new_a = copy.deepcopy(a)
        
        # Modify the third element of each entry with a first element value of 7
        new_a = modify_third_element_of_seven(new_a)
        
        # Select a random sublist
        sublist_idx = np.random.randint(len(new_a))
        # Select a random element in the sublist that doesn't have an index of 7
        element_idx = np.random.choice([i for i, x in enumerate(new_a[sublist_idx]) if x[0] != 7])
        # Modify its values
        new_a[sublist_idx][element_idx] = modify_element(new_a[sublist_idx][element_idx], range_dict)
        
        # Check if the new individual is valid
        if is_valid_individual(new_a):
            break
    return new_a

def modify_third_element_of_seven(a):
    """Modify the third element of each entry with a first element value of 7."""
    for sublist in a:
        for item in sublist:
            if item[0] == 7:
                item[2] = np.random.uniform(1000/60, 3000/60)
    return a

global range_dict
range_dict = {
        1: (10, 30),
        2: (20, 60),
        3: 30,
        4: (30, 60),
        5: (20, 60),
        6: (30, 60),
    }

global adj_matrix
adj_matrix = (
    (  0, 300, 360, 210, 530, 475, 500, 690),
    (300,   0, 380, 270, 230, 285, 200, 390),
    (360, 380,   0, 510, 230, 665, 580, 770),
    (210, 270, 510,   0, 470, 265, 450, 640),
    (530, 230, 230, 470,   0, 515, 360, 550),
    (475, 285, 665, 265, 515,   0, 460, 650),
    (500, 200, 580, 450, 360, 460,   0, 190),
    (690, 390, 770, 640, 550, 650, 190,   0))

import random
import copy
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import os
def simulated_annealing_maximize_v2(initial_temperature, alpha, num_iterations, a):
    global best_scores
    best_scores = [get_score(a)]
    
    current_a = copy.deepcopy(a)
    current_score = get_score(current_a)
    best_a = copy.deepcopy(current_a)
    best_score = current_score
    
    T = initial_temperature
    for i in range(num_iterations):
        neighbor_a = get_valid_neighbor(current_a)
        neighbor_score = get_score(neighbor_a)
        
        # Calculate delta E
        delta_E = neighbor_score - current_score
        if delta_E > 0 or np.random.rand() < math.exp(delta_E / T):
            current_a = copy.deepcopy(neighbor_a)
            current_score = neighbor_score
            if current_score > best_score:
                best_score = current_score
                best_a = copy.deepcopy(current_a)
        
        best_scores.append(best_score)  # 保存最佳得分
        
        # Reduce the temperature
        T *= alpha

    return best_a, best_score

print(get_score(best_a))
best_a, best_score = simulated_annealing_maximize_v2(1000, 0.995, 40000, best_a)

ani = FuncAnimation(fig, update, frames=range(1000), init_func=init, blit=True)
plt.show()

780.5943947937427


In [44]:
get_score(best_a)

783.1275304591754

In [39]:
a = [
    [[6, 40, 33.333333333333336], [1, 18, 33.333333333333336], [2, 60, 33.333333333333336], 
     [4, 37, 33.333333333333336], [3, 30, 33.333333333333336], [5, 21, 33.333333333333336], [7, float('inf'), 33.333333333333336]],

    [[2, 25, 33.333333333333336], [4, 32, 33.333333333333336], [3, 30, 33.333333333333336], 
     [5, 26, 33.333333333333336], [1, 10, 33.333333333333336], [6, 52, 33.333333333333336], [7, float('inf'), 33.333333333333336]],

    [[5, 33, 33.333333333333336], [3, 30, 33.333333333333336], [4, 53, 33.333333333333336], 
     [2, 34, 33.333333333333336], [1, 25, 33.333333333333336], [6, 32, 33.333333333333336], [7, float('inf'), 33.333333333333336]]
]

In [45]:
best_a

[[[6, 50, 42.04660619745327],
  [1, 30, 21.90859381137879],
  [2, 27, 48.335771999935574],
  [4, 51, 32.35820667004115],
  [3, 30, 29.945382536478753],
  [5, 23, 30.360165594870324],
  [7, inf, 49.02475400014204]],
 [[2, 35, 44.878966420309546],
  [4, 54, 25.51869130409665],
  [3, 30, 34.22636308906748],
  [5, 43, 31.454424549658814],
  [1, 28, 32.0200706318064],
  [6, 32, 32.374145489901075],
  [7, inf, 29.124701591658777]],
 [[5, 36, 39.76728138620567],
  [3, 30, 22.47610749814611],
  [4, 32, 29.374955952351513],
  [2, 20, 27.69765482884019],
  [1, 11, 35.136174969069785],
  [6, 42, 32.250570805526735],
  [7, inf, 29.2159882914974]]]